In [1]:
import pandas as pd
import numpy as np
import toml
from pathlib import Path
import polars as pl

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    df['county'] = df['county'].fillna('Outside Region')
    
    return df

In [4]:
# trip data
hh = pd.read_csv(output_path / 'agg/dash/hh_geog.csv')
# person data
person = pd.read_csv(output_path / 'agg/dash/pptyp_county.csv')
# worker data
employment = pd.read_csv(output_path / 'agg/dash/person_worker_type.csv')
# network data
df_network = load_network_summary(output_path / 'network/network_results.csv')
# job data
df_lu = pl.read_csv(output_path / 'landuse/parcels_urbansim.txt', separator=' ').to_pandas()

# Parcel geography
parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])
df_lu = df_lu.merge(parcel_geog, left_on='parcelid', right_on='ParcelID', how='left')

In [5]:
county_order = ['King','Kitsap','Pierce','Snohomish','Outside Region','Total']

person['person_county'] = pd.Categorical(person['person_county'], ordered=True,
                   categories=county_order)
hh['hh_county'] = pd.Categorical(hh['hh_county'], ordered=True,
                   categories=county_order)
employment['person_work_county'] = pd.Categorical(employment['person_work_county'], ordered=True,
                   categories=county_order)

df_network['county'] = pd.Categorical(df_network['county'], ordered=True,
                   categories=county_order)

## Households and Population

In [6]:
# Total Population
df_person = person[['person_county','psexpfac']].groupby('person_county', observed=True)['psexpfac'].sum()
df_hh = hh[['hh_county','hhexpfac']].groupby('hh_county', observed=True)['hhexpfac'].sum()

df = pd.concat([df_person,df_hh],axis=1)
df = df[df.index!='Outside Region']

df.loc['Total',] = df.sum()
df.rename(columns={'psexpfac': 'Persons',
                   'hhexpfac': 'Households'}, inplace=True)

df

,Persons,Households
King,"2,303,851","957,460"
Kitsap,"275,120","108,901"
Pierce,"925,555","350,141"
Snohomish,"851,485","319,624"
Total,"4,356,011","1,736,126"


## Workers and Jobs
By Workplace Location

Workers are Daysim outputs and Jobs are Parcel Inputs

In [7]:
# Jobs
df_jobs = df_lu[['emptot_p','CountyName']].groupby('CountyName').sum()[['emptot_p']]
df_jobs = df_jobs.reset_index()
df_jobs.rename(columns={'emptot_p': 'Jobs',
                   'CountyName':'County'}, inplace=True)
df_jobs.set_index('County')
df_jobs['Jobs'] = df_jobs['Jobs'].astype('float')

# workers
df = employment.groupby('person_work_county',observed=True)['psexpfac'].sum()

df = df.reset_index()
df.rename(columns={'psexpfac': 'Workers',
                   'person_work_county':'County'}, inplace=True)
df.set_index('County')
df['Workers'] = df['Workers'].astype('float')
df = df.merge(df_jobs, on='County')

df.loc['Total',:] = df.sum(axis=0, numeric_only=True)

df.loc['Total','County'] = 'Region'
df.reset_index(drop=True)

,County,Workers,Jobs
0,King,"1,440,729","1,442,506"
1,Kitsap,"104,852","98,821"
2,Pierce,"353,796","323,128"
3,Snohomish,"327,528","290,359"
4,Region,"2,226,905","2,154,814"


## Total VMT, VHT, Vehicle hours delay
Includes truck and external trips within bounds of the region (trips on partial links outside 4 counties are not included)

In [8]:
df_vmt = df_network.reset_index().groupby('county',observed=True)['VMT'].sum()
df_vht = df_network.reset_index().groupby('county',observed=True)['VHT'].sum()
df_delay = df_network.reset_index().groupby('county',observed=True)['total_delay'].sum()

df = pd.concat([df_vmt,df_vht,df_delay], axis=1)
# df = df[df.index!='Outside Region']
df.rename(columns={'total_delay': 'Total Delay Hours'}, inplace=True)

df.loc['Total',] = df.sum()
total_vmt = df.loc['Total','VMT']

df

,VMT,VHT,Total Delay Hours
county,,,
King,"43,165,825","1,363,418","149,519"
Kitsap,"4,328,161","127,671","2,648"
Pierce,"18,368,137","546,606","29,815"
Snohomish,"16,129,425","467,277","29,787"
Outside Region,"337,207","6,537",2
Total,"82,328,756","2,511,510","211,771"


## VMT by Type

In [9]:
vmt_df = pd.DataFrame()

# Daysim Resident Demand
df = pd.read_csv(output_path / 'agg/dash/person_vmt.csv')
df = df[df['mode'].isin(['SOV','HOV2','HOV3+']) & df['dorp']==1]
vmt_df.loc['Resident','VMT'] = df['travdist_wt'].sum()

# Trucks
df_network['medium_truck_vmt'] = df_network['@medium_truck']*df_network['length']
df_network['heavy_truck_vmt'] = df_network['@heavy_truck']*df_network['length']

vmt_df.loc['Medium Truck','VMT'] = df_network['medium_truck_vmt'].sum()
vmt_df.loc['Heavy Truck','VMT'] = df_network['heavy_truck_vmt'].sum()

# Externals
external_vmt = total_vmt - vmt_df['VMT'].sum()
vmt_df.loc['External and Other'] = external_vmt

vmt_df.loc['Total'] = total_vmt
vmt_df

,VMT
Resident,"62,532,985"
Medium Truck,"4,802,682"
Heavy Truck,"5,208,294"
External and Other,"9,784,795"
Total,"82,328,756"


## Vehicle Trips (Resident + External)

In [10]:
# Trips
df = pd.read_csv(output_path / 'trips_by_class.csv')
df.columns = ['class','trips','tod']
df['mode'] = df['class'].map({'sov_inc1': 'Drive Alone',
                 'sov_inc2': 'Drive Alone',
                 'sov_inc3': 'Drive Alone',
                 'hov2_inc1': 'Shared Ride',
                 'hov2_inc2': 'Shared Ride',
                 'hov2_inc3': 'Shared Ride',
                 'hov3_inc1': 'Shared Ride',
                 'hov3_inc2': 'Shared Ride',
                 'hov3_inc3': 'Shared Ride'})
df = df.groupby('mode').sum()[['trips']].reset_index()
df

,mode,trips
0,Drive Alone,"7,675,983"
1,Shared Ride,"3,167,635"


## Transit Boardings

In [11]:
# transit
transit = pd.read_csv(output_path / 'transit/daily_boardings_by_agency.csv')

df = transit[['agency_name','boardings']].set_index('agency_name').copy()
df.loc['Total',:] = df['boardings'].sum()
df.rename(columns={'boardings': 'Daily Boardings'}, inplace=True)

df


,Daily Boardings
agency_name,
King County Metro,"288,880"
Sound Transit,"140,400"
Community Transit,"26,595"
Pierce Transit,"23,561"
Kitsap Transit,"10,961"
Washington Ferries,"6,326"
Everett Transit,"5,097"
Total,"501,820"


## Mode Share 
Resident Trips

In [12]:
# mode share
mode_share = pd.read_csv(output_path / 'agg/dash/mode_share_county.csv')

df = mode_share.groupby('mode')['trexpfac'].sum().reset_index().set_index('mode')
df['Mode Share'] = (df['trexpfac']/ (mode_share['trexpfac'].sum()))
df.loc['Total', :] = df.sum(axis=0)
df['Mode Share'] = (df['Mode Share'] * 100).round(1)
df['Mode Share'] = df['Mode Share'].astype(str) + '%'
df['trexpfac'] = df['trexpfac'].apply(lambda x: f"{int(round(x)):,}")
df.rename(columns={'trexpfac': 'Total Person Trips'}, inplace=True)
df

,Total Person Trips,Mode Share
mode,,
Bike,"245,128",1.5%
HOV2,"3,477,484",21.3%
HOV3+,"2,298,569",14.1%
SOV,"7,249,939",44.5%
School Bus,"261,008",1.6%
TNC,"52,282",0.3%
Transit,"353,768",2.2%
Walk,"2,354,016",14.4%
Total,"16,292,194",100.0%


# Emissions
Daily Tons for light, medium, and heavy vehicles; bus vehicles are excluded.

In [13]:
df = pd.read_csv(output_path / 'emissions/emissions_summary.csv')

df = df[df['veh_type'].isin(['light','medium','heavy'])]
df = df.groupby('pollutant_name').sum()
df.rename(columns={'start_tons': 'Start', 'intrazonal_tons': 'Intrazonal', 'interzonal_tons': 'Interzonal',
                  'total_daily_tons': 'Total Daily'},
                  inplace=True)
df = df.loc[df.index.isin(['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent']),
            ['Start', 'Intrazonal', 'Interzonal', 'Total Daily']]

df

,Start,Intrazonal,Interzonal,Total Daily
pollutant_name,,,,
CO,124,2,235,361
CO2 Equivalent,"2,290",255,"37,735","40,281"
NOx,9,0,38,47
PM10 Total,0,0,5,5
PM25 Total,0,0,1,2


# Lane Miles

In [14]:
facility_type_dict = {
    1: 'Interstate',
    2: 'Other Freeway',
    3: 'Expressway',
    4: 'Ramp',
    5: 'Principal Arterial',
    6: 'Minor Arterial',
    7: 'Major Collector',
    8: 'Minor Collector',
    9: 'Local',
    10: 'Busway',
    11: 'Non-Motor',
    12: 'Light Rail',
    13: 'Commuter Rail',
    15: 'Ferry',
    16: 'Passenger-Only Ferry',
    17: 'Centroid Connector',
    18: 'Facility Connector',
    19: 'HOV Only Freeway',
    20: 'HOV Only Ramp',
    98: 'Weave Links'
}

ul3_dict = {
    0: 'Rail/Walk/Ferry',
    1: 'Freeway',
    2: 'Expressway',
    3: 'Urban Arterial',
    4: 'One-way Arterial',
    5: 'Centroid Connector',
    6: 'Rural Arterial'
}

county_dict = {
    33: 'King',
    35: 'Kitsap',
    53: 'Pierce',
    61: 'Snohomish'
}

In [15]:
# Select mid-day network
gdf = df_network.loc[df_network['tod'] == '10to14'].copy()
gdf['Lane Miles'] = gdf['length'] * gdf['num_lanes']

gdf['Facility Type'] = pd.Categorical(gdf['@facilitytype'].map(facility_type_dict), 
                                      ordered=True,
                                      categories=facility_type_dict.values())

df = gdf.groupby(['Facility Type'], observed=True)['Lane Miles'].sum().reset_index().set_index('Facility Type')

# df = df.drop(['@facilitytype','Facility Type'], axis=1)
df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
Facility Type,
Interstate,"1,468"
Other Freeway,767
Expressway,229
Ramp,537
Principal Arterial,"3,101"
Minor Arterial,"4,060"
Major Collector,"2,925"
Minor Collector,219
Local,299


In [16]:
gdf['Facility Group'] = pd.Categorical(gdf['data3'].map(ul3_dict), 
                                      ordered=True,
                                      categories=ul3_dict.values())

df = gdf.groupby(['Facility Group'], observed=True)['Lane Miles'].sum().reset_index().set_index('Facility Group')

df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
Facility Group,
Freeway,"2,236"
Expressway,677
Urban Arterial,"6,871"
One-way Arterial,452
Centroid Connector,"9,839"
Rural Arterial,"3,340"
Total,"23,415"


In [17]:
# Load link attributes and join
gdf['county'] = gdf['county'].replace(np.nan,'Outside Region')
df = gdf.groupby(['county'], observed=True)['Lane Miles'].sum().reset_index().set_index('county')

df.loc['Total','Lane Miles'] = df['Lane Miles'].sum()
df

,Lane Miles
county,
King,"10,803"
Kitsap,"2,022"
Pierce,"5,815"
Snohomish,"4,769"
Outside Region,58
Total,"23,468"


# Person Metrics

In [18]:
# Daysim data
import polars as pl
trip = pl.read_csv(r'../../../../outputs/daysim/_trip.tsv',separator='\t')
person = pl.read_csv(r'../../../../outputs/daysim/_person.tsv',separator='\t')
hh = pl.read_csv(r'../../../../outputs/daysim/_household.tsv', separator='\t')

### Average Daily Miles Driven per Person

In [19]:
if 'sov_ff_time' in trip.columns:
    drive_modes = [3, 4, 5]
    vehicle_trips = trip[['mode', 'dorp', 'travtime', 'sov_ff_time', 'travdist']].filter(pl.col("mode").is_in(drive_modes)).filter(pl.col("dorp") == 1)
    avg_vmt = vehicle_trips['travdist'].sum()/ person['psexpfac'].sum()
    print(f'Average Daily VMT per person, not including externals or trucks: {avg_vmt:.1f}')

### Hours of Congestion per Person per Year
For average Puget Sound resident:

In [20]:
if 'sov_ff_time' in trip.columns:
    drive_trips = trip[['mode', 'dorp', 'travtime', 'sov_ff_time', 'travdist']].filter(pl.col("mode").is_in(drive_modes))
    drive_trips = drive_trips.with_columns(
        delay=pl.col('travtime')-(pl.col('sov_ff_time')/100.0))
    minutes_to_hour = 60
    drive_mode_delay = summary_config['weekday_to_annual']*(drive_trips['delay'].sum()/person['psexpfac'].sum())/minutes_to_hour
    print(f'Annual hours of delay for residents, not including externals or trucks is: {drive_mode_delay:.1f}')

### Annual Hours of Delay by Average Truck
Average annual delay (hours) per truck trip in and through the region:

In [21]:
# Total truck trips

In [22]:
# Load truck trips
df = pd.read_csv(r'../../../../outputs/trucks/trucks_summary.csv',index_col=0)

# Truck delay
net_sum = pd.read_csv(r'../../../../outputs/network/delay_user_class.csv')

# Annual delay hours
daily_delay = net_sum[['@mveh','@hveh']].sum().sum()


# total truck trips
trips = df['prod'].sum()

# average annual delay hours per truck
x = (daily_delay*summary_config["weekday_to_annual"])/trips
print('{:0,.1f}'.format(x))

18.4


*Medium trucks only:*

In [23]:
x = (net_sum['@mveh'].sum()*summary_config["weekday_to_annual"])/df.loc['mt','prod']
print('{:0,.1f}'.format(x))

14.1


*Heavy trucks only:*

In [24]:
x = (net_sum['@hveh'].sum()*summary_config["weekday_to_annual"])/df.loc['ht','prod']
print ('{:0,.1f}'.format(x))

32.0


### % Population Walking or Biking for Transportation

In [25]:
trip_person = trip.join(person, on=["hhno", "pno"], how="left")
bike_walk_trips = trip_person.filter(pl.col("mode").is_in([1, 2]))

# Get unique persons with at least one bike/walk trip
bike_walk_persons = bike_walk_trips.select(["hhno", "pno"]).unique()
bike_walk_persons = bike_walk_persons.with_columns(bike_walk=pl.lit(True))

# Join back to all persons, mark bike_walk as False if not present
person_with_bike_walk = person.join(bike_walk_persons, on=["hhno", "pno"], how="left")
person_with_bike_walk = person_with_bike_walk.with_columns(
    pl.col("bike_walk").fill_null(False)
)

# Calculate share
share = (
    person_with_bike_walk.group_by("bike_walk")
    .agg(pl.col("psexpfac").sum())
    .with_columns(
        (pl.col("psexpfac") / pl.col("psexpfac").sum()).alias("share")
    )
)
print("Percent of population with at least one non-exercise walk or bike trip: {:.1%}".format(share.filter(pl.col("bike_walk") == True)["share"][0]))

Percent of population with at least one non-exercise walk or bike trip: 25.7%


## Household and Jobs within 1/4 mile transit

In [26]:
# Network data
df = pd.read_csv(r'../../../../outputs/transit/transit_access.csv',index_col=0)

**Households**

In [27]:
x = df.loc['hh_p','quarter_mile_transit']
print('{:,.0f}'.format(x) + (" households within 1/4 mile of transit"))
x = df.loc['hh_p','quarter_mile_transit']/df.loc['hh_p','total']
print('{:,.1%}'.format(x) + (" of total households"))

828,965 households within 1/4 mile of transit
47.7% of total households


**Jobs**

In [28]:
x = df.loc['emptot_p','quarter_mile_transit']
print('{:,.0f}'.format(x) + (" jobs within 1/4 mile of transit"))
x = df.loc['hh_p','quarter_mile_transit']/df.loc['emptot_p','total']
print('{:,.1%}'.format(x) + (" of total jobs"))

1,556,628 jobs within 1/4 mile of transit
38.5% of total jobs
